# NLE Assessed Coursework 3: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=184514 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/juliewe/resources')
sys.path.append(r'/Users/Bayley/Documents/resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords
import spacy
#nlp=spacy.load('en')
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import gutenberg

Sussex NLTK root directory is /Users/Bayley/Documents/resources


## Question 1: Part-of-Speech Tagging (25 marks)


In [3]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))
nlp_alice=list(nlp(alice).sents)

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [4]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [5]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]


**self.tagcounts**

tagcounts: A dictionary that stores all the part of speech (POS) tags that occur in the data along with how many times they each occur. For example, the tag `PROPN` occurs 1493 times in the trained data.

**self.wordcounts**

wordcounts: A dictionary that stores each word in the data, as the key, plus the number of occurences for each word in the data as the value. For example, the word `her` occurs 248 times in the data.

**self.tagperwordcounts**

tagperwordcounts: is a dictionary that stores each word that occurs in the data, as the key, and stores as the value a dictionary of all the part of speech (POS) tags for that word as the keys and the number of times that tag occurs for that word, as the value (stores tagcounts as the value in the dictionary). For example, we know the word `her` occurs 248 times, and is tagged as `DET` 158 times and as `PRON` 90 times. (158+90 = 248)    

b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [16]:
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        self.wordTList = {}
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                
    def tag(self):
        for word, dictionary in self.tagperwordcounts.items():
            for tag, count in dictionary.items():
                cnt_list = []
                cnt_list += [(tag, count)]
            countList = sorted(cnt_list, key=lambda tup: tup[1], reverse = True)
            self.wordTList[word] = self.wordTList.get(word, countList[0][0])
        return self.wordTList
#Evaluation
tagger2 = unigram_tagger(train)
tagged_train_data = tagger2.tag()


tt_dct = {}
for item in test:
    for pair in item:
        tt_dct[pair[0]] = tt_dct.get(pair[0], pair[1])
cnt = 0;
total = len(tt_dct)

for train_word, train_tag in tagged_train_data.items():
    for test_word, test_tag in tt_dct.items():
        if train_word == test_word and train_tag == test_tag:
            cnt += 1
accuracy = cnt/total
print("Accuracy: {:%}".format(accuracy))

Accuracy: 70.652174%


**TAG Method**

My tag method works in creating a dictionary of most likely tags. It starts by going through each word and dictionary within dictionary 'tagperwordcounts'. Here it goes through each tag and count within the inner dictionary and creates a list of all tags and counts, named cnt_list. The list of tags and counts is then sorted in desecending order (of count value). Next the first elememt in the sorted list of tags and counts pairs adding the value of the key (the word) into the wordTagList dictionary.

**Evaluation**

In the evaluation section (code under class labelled evaluation), the unigram tagger is run on the training data and then tags the training data. A dictionary is then made for the testing data, for each pair in the test data is added to the dictionary. This is stored as total.
The next section calculates the number of words that have been correctly tagged, along with calculating the accuracy of the tagger. My tagger is said to have an accuracy of ~71%. I believe that this should be higher as stated in lectures taggers have an accuracy of around 90%, this is most likely the case as the code may be testing each word type only once, with its most recent tag assigned. This is then a problem with how i constrcuted the test dictionary 'tt_dct' and the reason for not correcting this was due to it being found out late when looking over my code along with timing difficulties. If this was to be adjusted, accuracy should be much higher. To further improve, default or guess tags could be assigned if the test word is not seen in training data.

In the cell below, a `hmm_tagger` class is defined, which will be used in parts c and d of this question.

In [17]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

HMM taggers (Hidden Markov Models_ are used to incorporate information about likely tag sequences into a tagger. The reason these are used is due to word types being ambigious to PoS tag, for example in this sentence:
<br>`Every night i dream the same dream`<br>
In the sentence above, the word **dream** occurs twice, but should be labelled with different tags. The first occurence should be tagged verb, while the second occurence should be tagged noun. We know this as after pronouns i.e *I*, we would usually expect a verb, while after adjectives *same* we would expect a noun. We want to encode this information in order to ensure the correct tag is given, as mistakes would be made if we go with the most frequently occuring tag in this case.

**To calculate the probability of an observed sequence of words for a given sequence of tags** <br> $$P(w_1^n|t_1^n)$$ you would use the following formula:
![foward algorithm](fowardalg.png)
Also known as the foward algorithm. This calculates the probabilty of the words, given the tags. This is the number of times a word is given the tag, divided by the total amount of times the tag occurs. For example, the tag `PROPN` occurs **1493** times (shown in 1a), and the word `alice` is tagged as a PROPN **394 times** (alice': {'PROPN': 394}). Therefore the probability of the word is `alice` given the tag is PROPN is 
$\frac{394}{1493} = 0.2638981916$

We will know for the word sequence `like a wild beast`, calculate the probabilty of this word sequence given a tag sequence. *Note: the current word only depends on the current tag.* The tag sequence would be SCONJ, DET, ADJ, NOUN.
So the probability of the observed sequence given the sequence of tags is: <br><br>
$P(w_1^n|t_1^n)$ = $\frac{47}{674}$ * $\frac{624}{3548}$ * $\frac{2}{1389}$ * $\frac{1}{3295}$ = `5.35933952*10^-9`<br><br>
The more words and tags used, the smaller the probabilty tends to be. This is supported through if increase the previous sequence to`like a wild beast screamed, off with her head!`, the probability of the observed sequence given the sequence of tags (SCONJ, DET, ADJ, NOUN, VERB, PUNC, ADV, ADP, PRON, NOUN, PUNC) would be: <br><br>
$P(w_1^n|t_1^n)$ = $\frac{47}{674}$ * $\frac{624}{3548}$ * $\frac{2}{1389}$ * $\frac{1}{3295}$ * $\frac{4}{4609}$ * $\frac{2399}{7167}$ * $\frac{18}{2262}$ * $\frac{179}{2594}$ * $\frac{248}{3223}$ * $\frac{48}{3295}$ * $\frac{444}{7167}$ = `5.9366729 * 10 ^-20`
<br><br> This is because the current word only depends on the current tag. This is only used if we know the tag sequence and if we also know the word sequence we are interested in, as we can calculate the probability of a word sequence given the sequence of tags.



**How do we use HMM to find most likely tag sequence, when given the sequence of words?** 
![UsingHMM](usinghmm.png)
We apply Baye's rule. 
![Applying Bayes](applybayes.png)<br>
This states the most likely tag sequence is equal to: the probability of word sequence given tag sequence, times probability of the tag sequence all divided by probability of the word sequence. However the denominator can be dropped, since the probability of the word sequence does not affect which tag sequence maximises this probability.
![Dropping Denominator](denomdrop.png)<br>

**Simplifying Assumptions**
Here we would assume output independence since the current observation depends `ONLY` on current state.
![Assuming Output Independence](assumoutputinde.png)<br>
The equation above states that the probability is equal to the product over all words in a sequence of the probability of each individual word given each individual tag.
Now we need to make the first order of Markov assumption: Ciurrent state depends `ONLY` on previous state.
![Assuming First Order](firstorderassumption.png)<br>



So, calculate the probablity of a possible sequence of tags for a given sequence of words ,$$P(t_1^n|w_1^n)$$ we would use the following formula:
![probab](probtaggiven.png)

Using the tags ADV, ADP, DET the test sentence used will be `off with her` generated from the test sentences.
**DECODING EXAMPLE**
Using this sequence the possible tag sequences are :
1. ADV, ADP, DET
2. ADV, ADP, PRON
3. ADP, ADP, DET
4. ADP, ADP, PRON
<br>These are all possible as **off** can either be an ADV or and ADP, while **with** must be ADP, and **her** can either be a DET or a PRON. The observation probabilities is shown in a table below:
![Observation Table](tableob.png)<br>
Along with the transition probabilities shown below:
![Transition Probability Table](trantab.png)<br>


**1) P(ADV, ADP, DET | off with her)**<br>
`P(off with her | ADV ADP DET)` = P(off | ADV) * P(with | ADP) * P(her| DET) = 0.25 * 1 * 0.637 = `0.15925` <br>
`P(ADV ADP DET)` = P(ADV | START) * P(ADP | ADV) * P(DET | ADP) = 0.1159859 * 0.077368 * 0.464892 = `0.004172`<br>
`P(ADV, ADP, DET | off with her)` = 0.15925 * 0.004172 = `0.000664349`

**2) P(ADV, ADP, PRON | off with her)**<br>
`P(off with her | ADV ADP PRON)` = P(off | ADV) * P(with | ADP) * P(her| PRON) = 0.25 * 1 * 0.363 = `0.09075` <br>
`P(ADV ADP PRON)` = P(ADV | START) * P(ADP | ADV) * P(PRON | ADP) = 0.1159859 * 0.077368 * 0.138888889 = `0.001246328`<br>
`P(ADV, ADP, PRON | off with her)` = 0.09075 * 0.001246328 = `0.000113104`

**3) P(ADP, ADP, DET | off with her)**<br>
`P(off with her | ADP ADP DET)` = P(off | ADP) * P(with | ADP) * P(her| DET) = 0.75 * 1 * 0.637 = `0.47775` <br>
`P(ADP ADP DET)` = P(ADP | START) * P(ADP | ADP) * P(DET | ADP) = 0.027736 * 0.05054 * 0.464892 = `0.000651671`<br>
`P(ADP, ADP, DET | off with her)` = 0.47775 * 0.000651671 = `0.000311336`

**4) P(ADP, ADP, PRON | off with her)**<br>
`P(off with her | ADP ADP PRON)` = P(off | ADP) * P(with | ADP) * P(her| PRON) = 0.75 * 1 * 0.363 = `0.27225` <br>
`P(ADP ADP PRON)` = P(ADP | START) * P(ADP | ADP) * P(PRON | ADP) = 0.027736 * 0.05054 * 0.138888889 = `0.00019469`<br>
`P(ADP, ADP, PRON | off with her)` = 0.27225 * 0.00019469 = `5.30044E-05`

THEREFORE, the tag sequence with the highesr probability is **ADV, ADP, DET**

d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]


The Viterbi algorithm finds the best tag sequence without enumerating all possibilities. Does this by exploiting these two HMM assumptions:
1. probability of next state only depends on current state
2. probability of current output only depends on current state

<br>This is an example of dynamic programming:
- Recursively decomposing a problem into sub-problems
- Keep track of sub problems (tabulate) solutions to sub problems, to build up to the final solution.

**Viterbi Sub Problems** <br>
For an input sequence $w_1^n$, a sub-problem correspons to a part `(i,t)` where:
- `i` is the position in the sequence, i <n
- `t` is the current PoS tag

For the test sentence `off with her`:
- **off**: i = 1, t1 = ADV, t2 = ADP
- **with** i = 2, t1 = ADP
- **her** i = 3, t1 = DET, t2 = PRON

**Viterbi initialisation (step 1)**<br>
Using:
![Observation Table](tableob.png)
![Transition Probability Table](trantab.png)<br>

Where `i = 1`, wi = `off`<br>
**Sub-Problem 1: t1 = ADV**<br>
`V(1,ADV) = P(t1 = ADV)` = P(off | ADV) * P(ADV | START) = 0.25 * 0.1159859 = `0.02899647`<br>
**Sub-Problem 2: t1 = ADP**<br>
`V(1,ADP) = P(t1 = ADP)` = P(off | ADP) * P(ADP | START) = 0.75 * 0.027736 = `0.020801815`<br>

**Recursive Step (step 2)**

Where `i = 2`, wi = `with`<br>
**Sub-Problem 1: If Previous tag was ADV**<br>
`V(2,ADP) = P(t2 = ADP)` = V(1,ADV) * P(with | ADP) * P(ADP | ADV) = 0.02899647 * 1 * 0.0773677189 = `0.00224339073966212`<br>
**Sub-Problem 1: If Previous tag was ADP**<br>
`V(2,ADP) = P(t2 = ADP)` = V(1,ADP) * P(with | ADP) * P(ADP | ADP) = 0.020801815 * 1 * 0.0505401234567901 = `0.00105132632001643`<br>

Probability that previous tag was ADV > Probability that previous tag was ADP, so `V(2,ADP) = 0.00224339073966212`, if second tag is ADP, the first tag is ADV. WE know the second tag will be ADP since it is the only tag it can be. This means the first tag will be ADV.

**Recursive Step (step 3)**

Where `i = 3`, wi = `her`<br>
**Sub-Problem 1: If Previous tag was ADP**<br>
`V(3,DET) = P(t3 = DET)` = V(2,ADP) * P(her | DET) * P(DET | ADP) = 0.00224339073966212 * 0.637 * 0.464891975308641 <br>
P(t3 = DET) = `0.000664349182447354` <br>

**Sub-Problem 2: If Previous tag was ADP**<br>
`V(3,PRON) = P(t3 = PRON)` = V(2,ADP) * P(her | PRON) * P(PRON | ADP) = 0.00224339073966212 * 0.363 * 0.138888888888888 <br>
P(t3 = PRON) = `0.000113104283124631` <br>

We already know that the previous tag is ADP. The current tag would be DET as the `P(t3 = DET) > P(t3 = PRON)`, therefore probability `P(t3 = DET)` = **0.000664349182447354**

This gives us the most likely tag sequence ADV ADP, DET for the word sequence `off with her`. This is the same sequence generated in the 1c, so I would believe that this is correct. I can confirm the second tag is correct as it was the only possible tag. By using the technique of considering all possible solutions for word 1 when solving each sub-problem for word 2.

**Complexity**:<br>
Overall, the Viterbi algorithm is better than using the brute force method. For a brute force method the number of possible tag sequences is $k^{n}$ where k is size of the tagset and n is the number of word tokens. Compared to the Viterbi algorithm which has $k^{2}*n$ possible sequences. 

For small values of *k* and *n*, the brute force method is more efficient. i.e. if k = 2 and n = 3, the brute force method would have 2^3 possibilities (8), while Viterbi would have (2^2 * 3) possibilites (12). So for this case, Viterbi is slightly less efficient than tghe brute force method

For my case, I had 4 tags and 3 words, so using the brute force method I would have 4^3 (`64`) possibilities, while using the Viterbi complexity, 4^2 * 3 I would have only `48` Possibilites. So for the example I used, the Viterbi would have been more effiicent as it would only cycle through 48 out of the possible 64 possibilites, rather than all 64 in the brute force method.


In a more realistic example, the number of word tokens and size of tagsets are usually quite large, and as these increase, the number of sequences found by the brute force method increases exponentially. An still relatively small exampler could consist of 10 tags and 8 words in the sequence, which would have 100 million (10^8) possibilities in the brute force method, while only 800 (10^2 * 8) in the Viterbi method.